## Convolutional neural network 
- >Image classification
    -  split traindata and testdata
    -  normalize tensor [0-1] -> [-1-1]
    -  put train/test data to Dataloader(shuffle,batch_size,num_workers) respectively


### Import libraries

In [59]:
import torch 
import torch.nn as nn                                          # Activation Funciton , Algo
from torch.autograd import Variable   
import torch.utils.data as Data                                # Data loader 
from torchvision import datasets , transforms                  # Download dataset
import matplotlib.pyplot as plt 
from timer import timing
import numpy as nps

### Hyper Parameters
- > input image dim  [28,28,1]


In [66]:
INPUT_SIZE = 28   # 圖像尺寸 28x28x1
NUM_CLASSES = 10  # 10 分類數據
NUM_EPOCH = 50     # 訓練期數
BATCH_SIZE = 64   # 一次訓練 64 張圖片
LR = 0.01

### Dataset , DataLoader
- > dataset is wrapped by dataloader 
- > data dimension -> [ batch , 28  , 28 , 1 ]

In [65]:

train_dataset = datasets.MNIST(
    root = "./dataset/mnist",
    train = True,
    download = False,
    transform = transforms.ToTensor()
)
test_dataset = datasets.MNIST(
    root = "./dataset/mnist",
    train = False,
    download = False,
    transform = transforms.ToTensor()
)

train_dataloader = Data.DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True , 
    num_workers = 2 
)

test_dataloader = Data.DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    num_workers = 2,
    shuffle = True
)



### CNN 
- >conv1 (Conv2d , ReLU , Maxpool)    [Batch , 3 , 28 ,28]
- >conv2 (Conv2d , ReLU , Maxpool)    [Batch , 3 , 28 ,28]
- >output 
    

In [67]:
class CNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Sequential(   
            nn.Conv2d(
                in_channels= 1,
                out_channels= 16,           
                kernel_size= 5,                 
                stride= 1,
                padding= 2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)     
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,64,5,1,2),       
            nn.ReLU(),
            nn.MaxPool2d(2)     
        )
        self.out = nn.Linear(64 * 7 * 7 , 10)

    def forward(self , x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0) , -1)
        output = self.out(x)
        return output

CNN = CNN()
CNN

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=3136, out_features=10, bias=True)
)

In [29]:
a  = torch.rand(3,3)
print(a)
print('-'*100)
b = torch.max(a,1)[1]
print(b)

tensor([[0.0489, 0.0442, 0.4496],
        [0.7964, 0.2530, 0.3539],
        [0.7345, 0.9997, 0.2411]])
----------------------------------------------------------------------------------------------------
tensor([2, 0, 1])


In [32]:
x = torch.tensor([[1,2],[3,4]])
y = torch.tensor([[1,2],[3,3]])
ans = torch.eq(x,y).sum()
ans

tensor(3)

### Evaluate accuracy

In [68]:
def accuracy_good(predictions, labels):
    pred = torch.max(predictions.data , 1)[1]   # 取得 each row 最大值 index[1] => get index
    rights = torch.eq(labels.data , pred).sum() # .sum() 只算 Mapping 到 True的部分
    return rights/len(labels)*100



### Training

In [69]:
optimizer = torch.optim.Adam(CNN.parameters() , lr=LR)
loss_func = nn.CrossEntropyLoss() # for label model 

# Training and testing 
for epoch in range(NUM_EPOCH):
    
    train_right = []
    for step , (data , labels) in enumerate(train_dataloader):
        CNN.train()
        prediction = CNN(data)
        loss = loss_func(prediction , labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        right = accuracy_good(prediction,labels)
        train_right.append(right)

    accuracy = nps.average(train_right)
    print(f' EPOCH : {epoch}, AVERAGE ACCURACY => {accuracy}')


 EPOCH : 0, AVERAGE ACCURACY => 95.36913299560547
 EPOCH : 1, AVERAGE ACCURACY => 97.81449890136719
 EPOCH : 2, AVERAGE ACCURACY => 97.96109008789062
 EPOCH : 3, AVERAGE ACCURACY => 98.14932250976562
 EPOCH : 4, AVERAGE ACCURACY => 98.26759338378906
 EPOCH : 5, AVERAGE ACCURACY => 98.32756042480469
 EPOCH : 6, AVERAGE ACCURACY => 98.30590057373047
 EPOCH : 7, AVERAGE ACCURACY => 98.27091979980469
 EPOCH : 8, AVERAGE ACCURACY => 98.40585327148438
 EPOCH : 9, AVERAGE ACCURACY => 98.3292236328125
 EPOCH : 10, AVERAGE ACCURACY => 98.46414947509766
 EPOCH : 11, AVERAGE ACCURACY => 98.44915771484375
 EPOCH : 12, AVERAGE ACCURACY => 98.6107406616211
 EPOCH : 13, AVERAGE ACCURACY => 98.56243133544922
 EPOCH : 14, AVERAGE ACCURACY => 98.50746154785156
 EPOCH : 15, AVERAGE ACCURACY => 98.69569396972656
 EPOCH : 16, AVERAGE ACCURACY => 98.59574890136719
 EPOCH : 17, AVERAGE ACCURACY => 98.53578186035156
 EPOCH : 18, AVERAGE ACCURACY => 98.7490005493164
 EPOCH : 19, AVERAGE ACCURACY => 98.74234008